
# Callin Switzer
Clean the heavy/light flower metadata and save to file


In [1]:
import numpy as np
import os
import pandas as pd
import sys
import csv

from datetime import datetime

print(sys.version, "\n")

now = datetime.now()
print("last run on " + str(now))

3.6.1 |Anaconda custom (64-bit)| (default, May 11 2017, 13:04:09) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)] 

last run on 2017-12-07 21:29:32.201084


In [2]:
# define directories
baseDir = os.getcwd()
dataDir = '/Users/cswitzer/Dropbox/SonicationBehavior/SonBehData/SonicationBehavior_HeavyLight/'

In [3]:
baseDir

'/Users/cswitzer/Documents/GitRepos/BeeSonicationLearning'

### Clean bee metadata

In [4]:
metadataFile = "Raw_BeeMetadata_HeavyLightFlowers.csv"

# read in data
hlf = pd.read_csv(dataDir + metadataFile, header = 0, index_col = None)

In [5]:
hlf.head()

,dateFrozenOrMarked,trainFlowerPrepTime,trainStartTime,Hive,FlowerType1,FlowerType2,FlowerSwitchTime,switchRewardNum,trainingThreshold,BeeColorNum,IT,DateIT_Recorded,notes,trainNumRewardVisits,trialEndTime,trainingFile,notes.1
0,26-Sep-16,1418.0,NaN,3.0,solanum-glued,NaN,NaN,NaN,none,orange,3.71,NaN,NaN,NaN,NaN,/Users/callinswitzer/Dropbox/ExperSummer2016/B...,NaN
1,26-Sep-16,NaN,NaN,3.0,weighted-solanum-glued,NaN,NaN,NaN,none,orange,3.71,NaN,NaN,NaN,NaN,/Users/callinswitzer/Dropbox/ExperSummer2016/B...,same bee as above
2,26-Sep-16,1642.0,NaN,3.0,weighted-solanum-glued,NaN,NaN,NaN,none,gold,4.13,NaN,NaN,NaN,1651.0,NaN,NaN
3,26-Sep-16,1653.0,1703.0,4.0,sham-weighted_solanum_glued,NaN,NaN,NaN,NaN,white,3.38,NaN,NaN,NaN,1712.0,/Users/callinswitzer/Dropbox/ExperSummer2016/B...,NaN
4,26-Sep-16,1713.0,NaN,3.0,weighted,sham,NaN,24,none,gold,4.13,NaN,NaN,NaN,1732.0,/Users/callinswitzer/Dropbox/ExperSummer2016/B...,"at reward #24, the bee started buzzing on a no..."


In [6]:
# discard rows where I have noted that they should be discarded
discc = []
for ii in range(len(hlf)):
    sst = "".join(np.array(hlf.loc[ii,:]).astype(str)).lower()
    if "discard" in sst:
        discc.append(True)
    else:
        discc.append(False)

hlf["discardRow"] = discc

In [7]:
hlf2= hlf.loc[~hlf.discardRow, :]
hlf2.reset_index(inplace = True, drop=True)


In [8]:
# remove preliminary trials
rightFlowerType = [hlf2.FlowerType1[ii] in ["sham", "weighted"] for ii in range(len(hlf2))]

hlf3 = hlf2.loc[rightFlowerType, ]
hlf3 = hlf3.reset_index(drop=True)

In [9]:
# get training filenames (without paths)
tfl = ["/".join(hlf3.loc[ii, "trainingFile"].split("/")[-2:]) for ii in range(len(hlf3))]

In [10]:
hlf3["trainingFile_"] =tfl

In [11]:
hlf3.head()

,dateFrozenOrMarked,trainFlowerPrepTime,trainStartTime,Hive,FlowerType1,FlowerType2,FlowerSwitchTime,switchRewardNum,trainingThreshold,BeeColorNum,IT,DateIT_Recorded,notes,trainNumRewardVisits,trialEndTime,trainingFile,notes.1,discardRow,trainingFile_
0,26-Sep-16,1713.0,NaN,3.0,weighted,sham,NaN,24,none,gold,4.13,NaN,NaN,NaN,1732.0,/Users/callinswitzer/Dropbox/ExperSummer2016/B...,"at reward #24, the bee started buzzing on a no...",False,BeeGold_26Sept2016_Hive3_W_S/2016_09_26__17_14...
1,27-Sep-16,1032.0,1052.0,4.0,weighted,sham,1055,25,NaN,1,4.07,NaN,switchNum is first rew num after switching,NaN,1059.0,/Users/callinswitzer/Dropbox/ExperSummer2016/B...,frozen,False,Bee1_27Sept_Hive4_W_S/2016_09_27__10_52_05_amp...
2,27-Sep-16,1106.0,1155.0,4.0,sham,weighted,1157,35,NaN,2,4.42,NaN,NaN,NaN,1201.0,/Users/callinswitzer/Dropbox/ExperSummer2016/B...,NaN,False,Bee2_27Sept_Hive4_S_W/2016_09_27__11_54_44_amp...
3,28-Sep-16,826.0,832.0,3.0,sham,weighted,835,32,none,3,4.27,NaN,NaN,NaN,838.0,/Users/callinswitzer/Dropbox/ExperSummer2016/B...,NaN,False,Bee3_28Sept_hive3_S_W/2016_09_28__08_33_35_amp...
4,28-Sep-16,841.0,845.0,3.0,weighted,sham,847,45,none,4,4.27,NaN,sham flower not positioned very well (needle s...,NaN,NaN,/Users/callinswitzer/Dropbox/ExperSummer2016/B...,NaN,False,Bee4_28Sept2016_Hive3_W_S/2016_09_28__08_45_02...


In [12]:
hlf4 = hlf3.loc[:, ["BeeColorNum", "IT" ,"Hive", "FlowerType1", "FlowerType2", "trainingFile_", "dateFrozenOrMarked"]]
hlf4.Hive = hlf4.Hive.astype(int)

In [13]:
hlf4.head()

,BeeColorNum,IT,Hive,FlowerType1,FlowerType2,trainingFile_,dateFrozenOrMarked
0,gold,4.13,3,weighted,sham,BeeGold_26Sept2016_Hive3_W_S/2016_09_26__17_14...,26-Sep-16
1,1,4.07,4,weighted,sham,Bee1_27Sept_Hive4_W_S/2016_09_27__10_52_05_amp...,27-Sep-16
2,2,4.42,4,sham,weighted,Bee2_27Sept_Hive4_S_W/2016_09_27__11_54_44_amp...,27-Sep-16
3,3,4.27,3,sham,weighted,Bee3_28Sept_hive3_S_W/2016_09_28__08_33_35_amp...,28-Sep-16
4,4,4.27,3,weighted,sham,Bee4_28Sept2016_Hive3_W_S/2016_09_28__08_45_02...,28-Sep-16


In [14]:
hlf4.to_csv("/Users/cswitzer/Dropbox/SonicationBehavior/SonBehData/BeeMetaData/HeavyLight_BeeMetaData_cleaned.csv", header = True, index = False)

In [15]:
# print system info
import IPython
print(IPython.sys_info())

{'commit_hash': '5a894b9',
 'commit_source': 'installation',
 'default_encoding': 'UTF-8',
 'ipython_path': '/Users/cswitzer/anaconda/envs/python3/lib/python3.6/site-packages/IPython',
 'ipython_version': '5.3.0',
 'os_name': 'posix',
 'platform': 'Darwin-17.2.0-x86_64-i386-64bit',
 'sys_executable': '/Users/cswitzer/anaconda/envs/python3/bin/python',
 'sys_platform': 'darwin',
 'sys_version': '3.6.1 |Anaconda custom (64-bit)| (default, May 11 2017, '
                '13:04:09) \n'
                '[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]'}


In [16]:
# show installed packages and versions
!pip freeze 

alabaster==0.7.10
anaconda-client==1.6.3
anaconda-navigator==1.6.2
anaconda-project==0.6.0
appnope==0.1.0
appscript==1.0.1
asn1crypto==0.22.0
astroid==1.4.9
astropy==1.3.2
Babel==2.4.0
backports.functools-lru-cache==1.4
backports.shutil-get-terminal-size==1.0.0
beautifulsoup4==4.6.0
bitarray==0.8.1
blaze==0.10.1
bleach==1.5.0
bokeh==0.12.5
boto==2.46.1
Bottleneck==1.2.1
cffi==1.10.0
chardet==3.0.3
click==6.7
cloudpickle==0.2.2
clyent==1.2.2
colorama==0.3.9
conda==4.3.30
contextlib2==0.5.5
cryptography==1.8.1
cycler==0.10.0
Cython==0.25.2
cytoolz==0.8.2
dask==0.14.3
datashape==0.5.4
decorator==4.0.11
distributed==1.16.3
docutils==0.13.1
entrypoints==0.2.2
et-xmlfile==1.0.1
fastcache==1.0.2
Flask==0.12.2
Flask-Cors==3.0.2
gevent==1.2.1
greenlet==0.4.12
h5py==2.7.1
HeapDict==1.0.0
html5lib==0.999
idna==2.5
imagesize==0.7.1
ipykernel==4.6.1
ipython==5.3.0
ipython-genutils==0.2.0
ipywidgets==6.0.0
isort==4.2.5
itsdangerous==0.24
jdcal==1.3
jedi==0.10.2
Jinja2==2.9.6
jsonschema==2.6.0
jupyte

In [17]:
# convert to html, so ppl don't have to run python to see code
os.chdir(baseDir)
!jupyter nbconvert --to html 003_CleanHeavyLightMetaData

[NbConvertApp] Converting notebook 003_CleanHeavyLightMetaData.ipynb to html
[NbConvertApp] Writing 277610 bytes to 003_CleanHeavyLightMetaData.html
